In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from tqdm import tqdm
import numpy as np
from collections import defaultdict

In [ ]:
SHUFFLED_DIR  = '../00.data/00.wikidata/02.wikidata_shuffled/'
LANGUAGE_LIST = ['en', 'fr', 'de', 'es', 'it', 'pt', 'ko', 'ja']

In [ ]:
subjects = pq.read_table('../00.data/00.wikidata/01.wikidata_merged/subject_final.parquet').to_pandas()
properties = pq.read_table('../00.data/00.wikidata/01.wikidata_merged/property_final.parquet').to_pandas()
triples = pq.read_table('../00.data/00.wikidata/01.wikidata_merged/triples_final_final.parquet').to_pandas()

In [ ]:
triple_set = set(map(tuple, triples[['subject', 'property', 'object']].values))

# Subjects / Properties / Triples 파일 확인

In [ ]:
prop_subj_counts = triples.groupby('property')['subject'].nunique()
prop_obj_counts = triples.groupby('property')['object'].nunique()

prop_subj_alone = prop_subj_counts[prop_subj_counts == 1].index.tolist()
prop_obj_alone = prop_obj_counts[prop_obj_counts == 1].index.tolist()
bad_pids = sorted(set(prop_subj_alone) | set(prop_obj_alone))
mask_bad = triples["property"].isin(bad_pids)
triples = triples[~mask_bad].reset_index(drop = True)


In [ ]:
triples

In [ ]:
# 이미 정렬된 triples를 섞어서 상위 100개만 사용 (랜덤으로 100개 뽑은거랑 같은)
triples_sample = triples.sample(frac=1, random_state = 4129).reset_index(drop = True)

In [ ]:
triples_sample = triples_sample.iloc[:500]
triples_sample

# 1. **Subject Shuffle**

In [ ]:
np.random.seed(4129)

triples_base = triples_sample.copy()

# 기존 (subject, property, object) 관계 집합
# triple_set = set(map(tuple, triples[['subject', 'property', 'object']].values))

# 후보 subject 풀 (전체 분포 유지용)
subject_pool = triples['subject'].unique()

# 결과를 담을 배열
new_subjects = triples_base['subject'].to_numpy().copy()

unswappable_idx_subject = []  # 셔플 불가 index 기록
max_try = 50                  # 한 행당 최대 시도 횟수


for idx, (subj, prop, obj) in tqdm(
    enumerate(triples_base[['subject', 'property', 'object']].itertuples(index=False, name=None)),
    total=len(triples_base)
):
    chosen = None
    for _ in range(max_try):
        cand = np.random.choice(subject_pool)

        # 1) 같은 subject 그대로 나오면 패스 (원본 관계 유지 방지용)
        if cand == subj:
            continue

        # 2) 기존 triples에 존재하는 (cand, prop, obj)면 안 됨
        if (cand, prop, obj) in triple_set:
            continue

        chosen = cand
        break

    if chosen is None:
        # 이 행은 안전하게 못 바꾸겠다고 표시
        unswappable_idx_subject.append(idx)
        # 원본 subj 그대로 둠
    else:
        new_subjects[idx] = chosen

# DataFrame 반영
shuffled_subject = triples_base.copy()
shuffled_subject['subject'] = new_subjects

print("✅ Shuffled_SUBJECT 완료:", len(shuffled_subject))
print("🧾 셔플 불가 행 수:", len(unswappable_idx_subject))

# 검증: 원본 triple과 완전히 분리됐는지
orig_triplet = set(map(tuple, triples_base[['subject','property','object']].values))
new_triplet  = set(map(tuple, shuffled_subject[['subject','property','object']].values))
intersect = orig_triplet & new_triplet

print(f"🚫 원본과 동일한 triplet 수: {len(intersect)}")
if len(intersect) == 0:
    print("✅ 기존 triple 재등장 없음")
else:
    print("⚠️ 일부 기존 triple 유지됨 (대부분 unswappable 인덱스일 가능성 높음)")

In [ ]:
shuffled_subject

# 2. **Object Shuffle**

In [ ]:
np.random.seed(4129)

triples_base = triples_sample.copy()

# 기존 (subject, property, object) 관계 집합
# triple_set = set(map(tuple, triples[['subject', 'property', 'object']].values))

# 후보 subject 풀 (전체 분포 유지용)
object_pool = triples_base['object'].unique()

# 결과를 담을 배열
new_objects = triples_base['object'].to_numpy().copy()

unswappable_idx_object = []  # 셔플 불가 index 기록
max_try = 50                  # 한 행당 최대 시도 횟수


for idx, (subj, prop, obj) in tqdm(
    enumerate(triples_base[['subject', 'property', 'object']].itertuples(index=False, name=None)),
    total=len(triples_base)
):
    chosen = None
    for _ in range(max_try):
        cand = np.random.choice(object_pool)

        # 1) 같은 object 그대로 나오면 패스 (원본 관계 유지 방지용)
        if cand == obj:
            continue

        # 2) 기존 triples에 존재하는 (cand, prop, obj)면 안 됨
        if (subj, prop, cand) in triple_set:
            continue

        chosen = cand
        break

    if chosen is None:
        # 이 행은 안전하게 못 바꾸겠다고 표시
        unswappable_idx_object.append(idx)
        # 원본 subj 그대로 둠
    else:
        new_objects[idx] = chosen

# DataFrame 반영
shuffled_object = triples_base.copy()
shuffled_object['object'] = new_objects

print("✅ Shuffled_OBJECT 완료:", len(shuffled_object))
print("🧾 셔플 불가 행 수:", len(unswappable_idx_object))

# 검증: 원본 triple과 완전히 분리됐는지
orig_triplet = set(map(tuple, triples_base[['subject','property','object']].values))
new_triplet  = set(map(tuple, shuffled_object[['subject','property','object']].values))
intersect = orig_triplet & new_triplet

print(f"🚫 원본과 동일한 triplet 수: {len(intersect)}")
if len(intersect) == 0:
    print("✅ 기존 triple 재등장 없음")
else:
    print("⚠️ 일부 기존 triple 유지됨 (대부분 unswappable 인덱스일 가능성 높음)")


In [ ]:
shuffled_object

# 3. Property-Scoped Subject Shuffle

In [ ]:
np.random.seed(4129)

# tri_sample 기준으로 작업한다고 가정
triples_base = triples_sample.copy()

# 원본 triple 집합 (무결성 체크용)
# triple_set = set(map(tuple, triples[['subject', 'property', 'object']].values))

# 결과 DataFrame
shuffled_property_scoped_subject = triples_base.copy()

# 셔플 불가 인덱스 기록
unswappable_idx_property_scoped_subjects = []

property_scoped_subject = (
    triples.groupby('property')['subject']
    .apply(lambda s: s.unique().tolist())
    .to_dict()
)

# -------------------------------
# pid 그룹별로 qid 재배정 (Property-scoped subject shuffle)
# -------------------------------
for prop, group in tqdm(triples_base.groupby('property')):

    # 이 pid 그룹에서 사용 가능한 qid 후보들
    group_subjects = property_scoped_subject.get(prop, [])

    # pid 그룹 안에 qid가 1개뿐이면 전부 셔플 불가
    if len(group_subjects) <= 1:
        unswappable_idx_property_scoped_subjects.extend(group.index.tolist())
        continue

    # (pid, qsub)별로 원래 쓰였던 qid 집합 준비
    #  -> 같은 qsub 안에서는 기존에 쓰인 qid는 피하려고 하는 용도
    used_subjects_by_object = (
        group.groupby('object')['subject']
             .apply(lambda s: set(s.tolist()))
             .to_dict()
    )

    new_subjects = []
    
    for idx, row in group.iterrows():
        obj = row['object']

        # 현재 qsub에서 이미 쓰였던 qid는 후보에서 제외
        banned = used_subjects_by_object.get(obj, set())
        candidate_pool = [s for s in group_subjects if s not in banned]

        if not candidate_pool:
            # 이 행은 셔플 불가 → 원본 유지
            unswappable_idx_property_scoped_subjects.append(idx)
            new_subjects.append(row['subject'])
            continue

        # 후보 중에서 (qid, pid, qsub)이 원본 triple에 없는 것만 선택
        chosen = None
        for _ in range(3000):  # 최대 30번 시도
            cand = np.random.choice(candidate_pool)
            if (cand, prop, obj) not in triple_set:
                chosen = cand
                used_subjects_by_object.setdefault(obj, set()).add(cand)

                break

        if chosen is None:
            # 후보는 있지만 전부 원본 triple과 충돌 → 셔플 불가
            unswappable_idx_property_scoped_subjects.append(idx)
            new_subjects.append(row['subject'])
        else:
            new_subjects.append(chosen)

    # qid 교체
    shuffled_property_scoped_subject.loc[group.index, 'subject'] = new_subjects

print("✅ Shuffled_QID_by_PID 완료:", len(shuffled_property_scoped_subject))
print("🧾 셔플 불가 행 수:", len(unswappable_idx_property_scoped_subjects))


orig_triplet = triple_set
new_triplet = set(map(tuple, shuffled_property_scoped_subject[['subject', 'property', 'object']].values))

intersect = orig_triplet & new_triplet

print(f"🚫 원본과 동일한 triplet 수: {len(intersect)}")
if len(intersect) == 0:
    print("✅ 완전 무결성 유지 (기존 관계 재등장 없음)")
else:
    print("⚠️ 일부 기존 관계가 유지됨 — 셔플 불가 pid 그룹 비율 확인 필요")


In [ ]:
shuffled_property_scoped_subject

# 4. Property-Scoped Object Shuffle

In [ ]:
np.random.seed(4129)

# tri_sample 기준으로 작업한다고 가정
triples_base = triples_sample.copy()

# 원본 triple 집합 (무결성 체크용)
# triple_set = set(map(tuple, triples[['subject', 'property', 'object']].values))

# 결과 DataFrame
shuffled_property_scoped_object = triples_base.copy()

# 셔플 불가 인덱스 기록
unswappable_idx_property_scoped_objects = []

property_scoped_object = (
    triples.groupby('property')['object']
    .apply(lambda s: s.unique().tolist())
    .to_dict()
)

# -------------------------------
# pid 그룹별로 qid 재배정 (Property-scoped subject shuffle)
# -------------------------------
for prop, group in tqdm(triples_base.groupby('property')):

    # 이 pid 그룹에서 사용 가능한 qid 후보들
    group_objects = property_scoped_object.get(prop, [])

    # pid 그룹 안에 qid가 1개뿐이면 전부 셔플 불가
    if len(group_objects) <= 1:
        unswappable_idx_property_scoped_objects.extend(group.index.tolist())
        continue

    # (pid, qsub)별로 원래 쓰였던 qid 집합 준비
    #  -> 같은 qsub 안에서는 기존에 쓰인 qid는 피하려고 하는 용도
    used_objects_by_subject = (
        group.groupby('subject')['object']
             .apply(lambda s: set(s.tolist()))
             .to_dict()
    )

    new_objects = []
    
    for idx, row in group.iterrows():
        sbj = row['subject']

        # 현재 qsub에서 이미 쓰였던 qid는 후보에서 제외
        banned = used_objects_by_subject.get(sbj, set())
        candidate_pool = [o for o in group_objects if o not in banned]

        if not candidate_pool:
            # 이 행은 셔플 불가 → 원본 유지
            unswappable_idx_property_scoped_objects.append(idx)
            new_objects.append(row['object'])
            continue

        # 후보 중에서 (qid, pid, qsub)이 원본 triple에 없는 것만 선택
        chosen = None
        for _ in range(3000):  # 최대 30번 시도
            cand = np.random.choice(candidate_pool)
            if (sbj, prop, cand) not in triple_set:
                chosen = cand
                used_objects_by_subject.setdefault(sbj, set()).add(cand)

                break

        if chosen is None:
            # 후보는 있지만 전부 원본 triple과 충돌 → 셔플 불가
            unswappable_idx_property_scoped_objects.append(idx)
            new_objects.append(row['object'])
        else:
            new_objects.append(chosen)

    # object 교체
    shuffled_property_scoped_object.loc[group.index, 'object'] = new_objects

print("✅ Shuffled_PID_by_QID 완료:", len(shuffled_property_scoped_object))
print("🧾 셔플 불가 행 수:", len(unswappable_idx_property_scoped_objects))


orig_triplet = triple_set
new_triplet = set(map(tuple, shuffled_property_scoped_object[['subject', 'property', 'object']].values))

intersect = orig_triplet & new_triplet

print(f"🚫 원본과 동일한 triplet 수: {len(intersect)}")
if len(intersect) == 0:
    print("✅ 완전 무결성 유지 (기존 관계 재등장 없음)")
else:
    print("⚠️ 일부 기존 관계가 유지됨 — 셔플 불가 pid 그룹 비율 확인 필요")


## **original + 4shuffle 파일들의 인덱스 일치여부 확인 셀**

In [ ]:
triples_sample

In [ ]:
shuffled_subject

In [ ]:
shuffled_object

In [ ]:
shuffled_property_scoped_subject

In [ ]:
shuffled_property_scoped_object

In [ ]:
triples = pq.read_table('../00.data/00.wikidata/01.wikidata_merged/triples_filtered.parquet').to_pandas()
triple_set = set(map(tuple, triples[['subject', 'property', 'object']].values))
def count_in_triple_set(df, name="df"):
    """
    df: subject, property, object 컬럼을 가진 DataFrame
    name: 출력용 이름
    """
    # 각 행을 (subject, property, object) 튜플로 변환
    tup_list = list(map(tuple, df[['subject', 'property', 'object']].values))
    
    # triple_set 안에 있는 튜플 개수 카운트
    cnt = sum(t in triple_set for t in tup_list)
    total = len(df)
    print(f"{name}: {cnt} / {total} ({cnt/total:.6%}) triples 가 triple_set 안에 있음")
    return cnt

# 각 셔플 결과에 대해 카운트
count_in_triple_set(shuffled_subject, "shuffled_subject")
count_in_triple_set(shuffled_object, "shuffled_object")
count_in_triple_set(shuffled_property_scoped_subject, "shuffled_property_scoped_subject")
count_in_triple_set(shuffled_property_scoped_object, "shuffled_property_scoped_object")

In [ ]:
triples_sample.to_parquet(f"{SHUFFLED_DIR}00.original_500.parquet", compression = "snappy", index = True)
shuffled_subject.to_parquet(f"{SHUFFLED_DIR}01.subject_shuffled_500.parquet", compression = "snappy", index = True)
shuffled_object.to_parquet(f"{SHUFFLED_DIR}02.object_shuffled_500.parquet", compression = "snappy", index = True)
shuffled_property_scoped_subject.to_parquet(f"{SHUFFLED_DIR}03.property_scoped_subject_shuffled_500.parquet", compression = "snappy", index = True)
shuffled_property_scoped_object.to_parquet(f"{SHUFFLED_DIR}04.property_scoped_object_shuffled_500.parquet", compression = "snappy", index = True)